In [1]:
import random

We will have a graph with nodes (intersections), edges (roads). 

Assumptions:
- random routing
- uniform car distribution?
- no time spent in intersection
- cars are terminated at nodes which have no outlet

In [1255]:


class Map:
    def __init__(self, intersections, roads): # units of length?
        self.intersections = intersections
        self.roads = roads

    def update(self):
        for road in self.roads:
            road.update()

class Intersection:
    def __init__(self, roads=[]):
        self.roads = roads # in what order ?
        self.terminations = 0

        self.get_out_roads()

    def add_road(self, road):
        self.roads.append(road)
        self.get_out_roads()

    def get_out_roads(self):
        self.out_roads = []
        for road in self.roads:
            if road.get_start() == self:
                self.out_roads.append(road)

    def add_cars(self, num_cars, in_road): # cars incoming from a segment: we have already checked that the 
        # intersection can hold this
        
        # calculate how many cars to send (make this depend on a direction dictionary)
        #direction sent based on in_road

        if len(self.out_roads) != 0: # for now, the direction is just random
            choice = random.choice(self.out_roads)
            
            choice.add_cars(num_cars)# choose random road
                
        else:
            self.terminations += num_cars
            
        # else cars just disappear

    def max_in_flux(self, in_segment=None,):
        return 500 # this will depend on the percentages of where they are headed (from direction dict)
        # this is a linear equation thing that I need to figure out




class Road:
    def __init__(self, dt, start, end, num_cars, speed_limit, length, lanes, name):
        self.dt = dt # size of time step
        self.start = start # starting intersection
        self.end = end # ending intersection
        self.num_cars = num_cars # number of cars
        self.speed_limit = speed_limit
        self.length = length # length of road (integer)
        self.lanes = lanes # number of lanes
        self.name = name
        self.cars = [self.num_cars / self.length for i in range(self.length)] # start with uniform distribution: is this variable ever needed again?


        
        self.connect()
        self.speed = 0
        self.update_speed()
        self.init_segments()

    def init_segments(self):
        self.segments = []
        self.segments = [RoadSegment(self, i) for i in self.cars]
        if len(self.segments) == 1:
            self.segments[0].set_adjacent(self.start, self.end)

        else:
            self.segments[0].set_adjacent(self.start, self.segments[1])
            for i in range(1, len(self.segments)-1):
                self.segments[i].set_adjacent(self.segments[i-1],self.segments[i+1])

            self.segments[-1].set_adjacent(self.segments[-2],self.end)
        

    def connect(self):
        self.start.add_road(self)
        self.end.add_road(self)

    def update_speed(self): # I need to re-implement this I think
        if self.num_cars > 0:
            self.speed =  self.speed_limit / self.num_cars
        else:
            self.speed = self.speed_limit
    
    def get_start(self):
        return self.start
    
    def update(self):
        for i in self.segments[::-1]: # do this in reverse
            i.update()
        self.num_cars = 0
        for i in self.segments:
            self.num_cars += i.num_cars
        bar = ""
        for i in range(int(self.num_cars)):
            bar += "#"

        print(self.name, bar)
        # send number of cars into intersection based on uniform density of cars

    def add_cars(self, num_cars):
        self.segments[0].add_cars(num_cars)
    def remove_cars(self, num_cars):
        self.segments[-1].remove_cars(num_cars)

class RoadSegment:
    def __init__(self, parent, num_cars, next_segment=None, prev_segment=None):
        self.num_cars = num_cars
        self.next_segment = next_segment
        self.prev_segment = prev_segment
        self.capacity = 100 # this is the max capacity of the road
        self.parent = parent

    def set_adjacent(self, prev_segment, next_segment):
        self.prev_segment = prev_segment
        self.next_segment = next_segment

    def add_cars(self, num_cars, in_road=None): # dummy variable so that we overload function name for intersection
        self.num_cars += num_cars

    def remove_cars(self, num_cars):
        self.num_cars -= num_cars

    def max_in_flux(self): # returns how many cars we can be sent
        return self.capacity - self.num_cars 
        
    def update(self):
        out_flux = min(self.next_segment.max_in_flux(), self.num_cars)
        self.remove_cars(out_flux)
        self.next_segment.add_cars(out_flux, in_road=self.parent)




In [1256]:
delta_time = 1
inter1 = Intersection()
inter2 = Intersection()
inter3 = Intersection()
#   #---\
#   |    #------------#(4)
#   #--/
inter4 = Intersection()
road1 = Road(dt=delta_time, start=inter1, end=inter2, num_cars=20, speed_limit=100, length=100, lanes=1, name="1")
road2 = Road(dt=delta_time, start=inter2, end=inter3, num_cars=20, speed_limit=100, length=100, lanes=1, name="2")
road3 = Road(dt=delta_time, start=inter3, end=inter1, num_cars=20, speed_limit=100, length=100, lanes=1, name="3")
road4 = Road(dt=delta_time, start=inter2, end=inter4, num_cars=20, speed_limit=100, length=100, lanes=1, name="4")

In [1257]:
map = Map(intersections = [inter1, inter2, inter3, inter4], roads=[road1, road2, road3, road4])

In [1309]:
map.update()
print("Terminations: ", inter4.terminations)

1 ###################
2 ################
3 ###################
4 #############
Terminations:  10.399999999999995


Next Steps:
- Make a better model wrt choosing how many cars move into the intersection (uniform distribution is bad)
- Make it easier to make a Map object (better instantiation of roads, intersections)
- Model how the next road affects the previous road
- Add visualization
- max number of cars on each segment
